In [40]:
import sys 
sys.path.append("..") 
import torch
import torch.nn as nn

In [41]:
import os
print("Current working directory:", os.getcwd())
print("Files here:", os.listdir(".")[:20])


Current working directory: /Users/setti/Desktop/Battery_PINN_SOH/notebooks
Files here: ['.ipynb_checkpoints', '01_pinn_soh_replication.ipynb']


In [42]:
from src.models import FNet, GNet

fnet = FNet()
gnet = GNet()

fnet, gnet


(FNet(
   (net): Sequential(
     (0): Linear(in_features=17, out_features=64, bias=True)
     (1): ReLU()
     (2): Linear(in_features=64, out_features=64, bias=True)
     (3): ReLU()
     (4): Linear(in_features=64, out_features=1, bias=True)
   )
 ),
 GNet(
   (net): Sequential(
     (0): Linear(in_features=35, out_features=64, bias=True)
     (1): ReLU()
     (2): Linear(in_features=64, out_features=64, bias=True)
     (3): ReLU()
     (4): Linear(in_features=64, out_features=1, bias=True)
   )
 ))

In [43]:
import importlib
import src.models
importlib.reload(src.models)

from src.models import FNet, GNet


In [44]:
fnet = FNet()
gnet = GNet()

fnet, gnet


(FNet(
   (net): Sequential(
     (0): Linear(in_features=17, out_features=64, bias=True)
     (1): ReLU()
     (2): Linear(in_features=64, out_features=64, bias=True)
     (3): ReLU()
     (4): Linear(in_features=64, out_features=1, bias=True)
   )
 ),
 GNet(
   (net): Sequential(
     (0): Linear(in_features=35, out_features=64, bias=True)
     (1): ReLU()
     (2): Linear(in_features=64, out_features=64, bias=True)
     (3): ReLU()
     (4): Linear(in_features=64, out_features=1, bias=True)
   )
 ))

In [45]:
import importlib
import src.losses
importlib.reload(src.losses)

from src.losses import data_loss
import torch

u_hat = torch.tensor([[0.95],[0.90]])
u_true = torch.tensor([[1.00],[0.88]])

print(data_loss(u_hat, u_true))


L_total: 0.5231143832206726 L_data: 0.5071816444396973 L_PDE: 0.0038039202336221933 L_mono: 0.012128839269280434
tensor(0.0015)


In [46]:
import importlib
import src.losses
importlib.reload(src.losses)

from src.models import FNet
from src.losses import gradients
import torch

B = 3
t = torch.rand(B, 1, requires_grad=True)
x = torch.rand(B, 16, requires_grad=True)

fnet = FNet()
u_hat = fnet(t, x)              # (B,1)

u_t, u_x = gradients(u_hat, t, x)

print("u_hat:", u_hat.shape)
print("u_t  :", u_t.shape)
print("u_x  :", u_x.shape)


L_total: 0.3304150104522705 L_data: 0.3226698040962219 L_PDE: 0.0031377444975078106 L_mono: 0.004607463721185923
u_hat: torch.Size([3, 1])
u_t  : torch.Size([3, 1])
u_x  : torch.Size([3, 16])


In [47]:
import importlib
import src.losses
importlib.reload(src.losses)

from src.models import FNet, GNet
from src.losses import gradients, pde_loss
import torch

B = 4
t = torch.rand(B, 1, requires_grad=True)
x = torch.rand(B, 16, requires_grad=True)

fnet = FNet()
gnet = GNet()

u_hat = fnet(t, x)
u_t, u_x = gradients(u_hat, t, x)

lpde = pde_loss(gnet, t, x, u_hat, u_t, u_x)
print("L_PDE:", lpde)


L_total: 0.27010098099708557 L_data: 0.25905051827430725 L_PDE: 0.000683657533954829 L_mono: 0.010366799309849739
L_PDE: tensor(0.0050, grad_fn=<MeanBackward0>)


In [48]:
import importlib
import src.losses
importlib.reload(src.losses)

from src.losses import mono_loss
import torch

# Case 1: decreasing SOH (should be 0)
u1 = torch.tensor([1.0, 0.99, 0.97, 0.95])
print("mono (decreasing):", mono_loss(u1))

# Case 2: has an increase (should be > 0)
u2 = torch.tensor([1.0, 0.98, 0.985, 0.96])
print("mono (has increase):", mono_loss(u2))


L_total: 0.17468257248401642 L_data: 0.15975339710712433 L_PDE: 0.0004282320733182132 L_mono: 0.014500941149890423
mono (decreasing): tensor(0.)
mono (has increase): tensor(0.0017)


In [49]:
import torch
from src.models import FNet, GNet
from src.losses import gradients, pde_loss, mono_loss, total_loss

# models
fnet = FNet()
gnet = GNet()

# optimizer (updates both nets)
opt = torch.optim.Adam(list(fnet.parameters()) + list(gnet.parameters()), lr=1e-3)

# fake batch (we'll replace with real dataset later)
B = 8
t = torch.rand(B, 1, requires_grad=True)
x = torch.rand(B, 16, requires_grad=True)
u_true = torch.rand(B, 1)  # fake labels

# forward
u_hat = fnet(t, x)
u_t, u_x = gradients(u_hat, t, x)

lpde = pde_loss(gnet, t, x, u_hat, u_t, u_x)
lmono = mono_loss(u_hat)  # placeholder; true mono needs sequences

L, ldata = total_loss(u_hat, u_true, lpde, lmono, alpha=1.0, beta=1.0)

# backward + update (THIS is the core training ritual)
opt.zero_grad()
L.backward()
opt.step()

print("L_total:", float(L), "L_data:", float(ldata), "L_PDE:", float(lpde), "L_mono:", float(lmono))


L_total: 0.5133945345878601 L_data: 0.4844096302986145 L_PDE: 0.0125201977789402 L_mono: 0.016464725136756897


In [50]:
Battery_PINN_SOH/src/losses.py


NameError: name 'Battery_PINN_SOH' is not defined

In [51]:
import importlib
import src.losses
importlib.reload(src.losses)

from src.losses import gradients, pde_loss, mono_loss, total_loss


L_total: 0.2923888564109802 L_data: 0.2737092971801758 L_PDE: 0.0020677505526691675 L_mono: 0.01661182940006256


In [52]:
def rul_from_soh(soh_pred, eol_threshold=0.8):
    """
    soh_pred: 1D tensor/list of predicted SOH over cycles for ONE cell: length T
    returns: RUL for each cycle (length T) and k_eol (int)
    """
    import torch
    soh_pred = torch.as_tensor(soh_pred).flatten()
    below = (soh_pred <= eol_threshold).nonzero(as_tuple=False)

    if len(below) == 0:
        # never reached EOL in the observed window
        k_eol = len(soh_pred) - 1
    else:
        k_eol = int(below[0].item())

    cycles = torch.arange(len(soh_pred))
    rul = k_eol - cycles
    return rul, k_eol


In [53]:
from src.dataset_csv import PairedCycleCSVDataset, infer_input_dim

csvs = [
  "/Users/setti/Desktop/Battery_PINN_SOH/data/2C_battery-1.csv",
  "/Users/setti/Desktop/Battery_PINN_SOH/data/2C_battery-2.csv",
]

ds = PairedCycleCSVDataset(csvs, target_col="capacity", add_cycle_index=True, normalize_x=True)

print("N samples:", len(ds))
x1, x2, y1, y2 = ds[0]
print("x1 shape:", x1.shape, "x2 shape:", x2.shape)
print("y1 shape:", y1.shape, "y2 shape:", y2.shape)
print("input_dim:", x1.numel())


N samples: 724
x1 shape: torch.Size([17]) x2 shape: torch.Size([17])
y1 shape: torch.Size([1]) y2 shape: torch.Size([1])
input_dim: 17


In [54]:
import pandas as pd
import numpy as np

csv_path = "/Users/setti/Desktop/Battery_PINN_SOH/data/2C_battery-1.csv"   # one of the ones you used

df = pd.read_csv(csv_path)
# Match paper dataloader.py behavior: insert cycle index THEN clean rows



# separate X and y exactly like the dataset
X_df = df.drop(columns=["capacity"])

# 1) columns with any NaN/Inf
bad_nan = X_df.columns[~np.isfinite(X_df.to_numpy()).all(axis=0)]
print("Columns with NaN/Inf:", list(bad_nan))

# 2) columns with zero range (xmax == xmin)
xmin = X_df.min(axis=0, numeric_only=True)
xmax = X_df.max(axis=0, numeric_only=True)
zero_range = (xmax - xmin) == 0
print("Zero-range columns:", list(X_df.columns[zero_range.values]))

# 3) show ranges for all columns (helps spot weird stuff)
ranges = (xmax - xmin).sort_values()
print(ranges.head(10))


Columns with NaN/Inf: ['voltage entropy']
Zero-range columns: []
voltage slope       0.000048
current slope       0.000401
voltage std         0.001003
voltage mean        0.001308
current std         0.006340
current mean        0.013909
voltage skewness    0.016537
CC Q                0.022000
voltage kurtosis    0.027790
CV Q                0.035000
dtype: float64


In [33]:
import sys
from pathlib import Path

# adjust if your notebook is in /notebooks
sys.path.append(str(Path("..").resolve()))

from src.dataset_csv import PairedCycleCSVDataset

csvs = [
    "/Users/setti/Desktop/Battery_PINN_SOH/data/2C_battery-1.csv",
    
]

ds = PairedCycleCSVDataset(csvs, target_col="capacity", add_cycle_index=True, normalize_x=True)

print("N samples:", len(ds))
x1, x2, y1, y2 = ds[0]
print("x1 shape:", x1.shape, "x2 shape:", x2.shape)
print("y1 shape:", y1.shape, "y2 shape:", y2.shape)
print("input_dim:", x1.numel())


N samples: 374
x1 shape: torch.Size([17]) x2 shape: torch.Size([17])
y1 shape: torch.Size([1]) y2 shape: torch.Size([1])
input_dim: 17


/Users/setti/Desktop/Battery_PINN_SOH/notebooks/../src/dataset_csv.py:139: RuntimeWarning: invalid value encountered in subtract
  X01 = (X - xmin) / (xmax - xmin)  # [0,1]
/Users/setti/Desktop/Battery_PINN_SOH/notebooks/../src/dataset_csv.py:139: RuntimeWarning: invalid value encountered in divide
  X01 = (X - xmin) / (xmax - xmin)  # [0,1]


In [34]:
import src.dataset_csv as d
print(d.__file__)


/Users/setti/Desktop/Battery_PINN_SOH/notebooks/../src/dataset_csv.py


In [35]:
import importlib, src.dataset_csv as d
importlib.reload(d)
print("reloaded:", d.__file__)


reloaded: /Users/setti/Desktop/Battery_PINN_SOH/notebooks/../src/dataset_csv.py


In [60]:
import pandas as pd
import numpy as np

csv_path = "/Users/setti/Desktop/Battery_PINN_SOH/data/2C_battery-1.csv"   # one of the ones you used

df = pd.read_csv(csv_path)
# Match paper dataloader.py behavior: insert cycle index THEN clean rows



# separate X and y exactly like the dataset
X_df = df.drop(columns=["capacity"])

# 1) columns with any NaN/Inf
bad_nan = X_df.columns[~np.isfinite(X_df.to_numpy()).all(axis=0)]
print("Columns with NaN/Inf:", list(bad_nan))

# 2) columns with zero range (xmax == xmin)
xmin = X_df.min(axis=0, numeric_only=True)
xmax = X_df.max(axis=0, numeric_only=True)
zero_range = (xmax - xmin) == 0
print("Zero-range columns:", list(X_df.columns[zero_range.values]))

# 3) show ranges for all columns (helps spot weird stuff)
ranges = (xmax - xmin).sort_values()
print(ranges.head(10))


Columns with NaN/Inf: ['voltage entropy']
Zero-range columns: []
voltage slope       0.000048
current slope       0.000401
voltage std         0.001003
voltage mean        0.001308
current std         0.006340
current mean        0.013909
voltage skewness    0.016537
CC Q                0.022000
voltage kurtosis    0.027790
CV Q                0.035000
dtype: float64


In [57]:
import importlib, src.dataset_csv as d
print("USING:", d.__file__)
importlib.reload(d)


USING: /Users/setti/Desktop/Battery_PINN_SOH/notebooks/../src/dataset_csv.py


<module 'src.dataset_csv' from '/Users/setti/Desktop/Battery_PINN_SOH/notebooks/../src/dataset_csv.py'>

In [71]:
import importlib, src.dataset_csv as d
print(d.__file__)
print("delete_3_sigma has to_numeric:", "to_numeric" in d.delete_3_sigma.__code__.co_names)


/Users/setti/Desktop/Battery_PINN_SOH/notebooks/../src/dataset_csv.py
delete_3_sigma has to_numeric: True


In [72]:
import src.dataset_csv as d, inspect
print(inspect.getsource(d.delete_3_sigma))


def delete_3_sigma(df: pd.DataFrame) -> pd.DataFrame:
    # Make sure "nan"/"inf"/"" strings become real NaN
    df = df.apply(pd.to_numeric, errors="coerce")

    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    df = df.reset_index(drop=True)

    out_index = []
    for col in df.columns:
        idx = _3_sigma(df[col].to_numpy())
        out_index.extend(idx.tolist())

    out_index = list(set(out_index))
    df = df.drop(out_index, axis=0)
    df = df.reset_index(drop=True)
    return df



In [69]:
!python train.py \
  --data_dir /Users/setti/Desktop/Battery_PINN_SOH/data \
  --pattern "2C_battery-2.csv" \
  --normalization \
  --norm_method minmax \
  --epochs 50 \
  --batch_size 64


python: can't open file '/Users/setti/Desktop/Battery_PINN_SOH/notebooks/train.py': [Errno 2] No such file or directory


In [70]:
cd /Users/setti/Desktop/Battery_PINN_SOH
python train.py \
  --data_dir /Users/setti/Desktop/Battery_PINN_SOH/data \
  --pattern "2C_battery-2.csv" \
  --normalization \
  --norm_method minmax \
  --epochs 50 \
  --batch_size 64


SyntaxError: invalid syntax (3949338866.py, line 2)